In [1]:
#COUPLED BALANCES

In [2]:
#isolate energy or mass balance. Reasonable results? Try having different temperatures during ta and tb
# look over excel. reasonable?
# forgot to remove packing volume from total reactor volume

In [3]:
#Ali recommends--

#compare MB with coupled
#
#
#use high pulse for dim #'s'
#
#
#
#


In [4]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import scipy.optimize as sopt
import pandas as pd
from pyomo.environ import *
from idaes.core.solvers import get_solver

In [5]:
# from PyDBDdim.utils import DatasetProcess
# DBD = DatasetProcess(r'Mesbah-DBD')
# DBD.folder_col
# time_col, _= DBD.current_segment()
# time_col = np.array(time_col)
# t_index = np.array(_)
# Vipab, Qab = DBD.get_two_region_VI()
# print(Vipab.shape)

In [6]:
df = pd.read_excel(r"data_col_5cm_23mm_dbd.xlsx")
#flowrate = np.array(df.flowrate_percent, dtype = 'float64')#it is in percents, we convert only when indexed below
#print(type(flowrate))
width_ns = np.array(df.width_ns, dtype = 'float64')*(10**-9) #ns to s
flowrate = np.ones((width_ns.shape))*0.65
freq = np.array(df.freq_kHz, dtype = 'float64')*1000 #kHz to Hz
voltage = np.array(df.voltage_kV, dtype = 'float64')*1000 #kV to V
#temp_C = np.array(df.temp_C, dtype = 'float64') +273 #now in Kelvin!
Tg = 300
temp_C = np.ones((freq.shape))*Tg
#power = np.array(df.power_w, dtype = 'float64')
Te = np.array(df.iloc[:,9], dtype = 'float64')*(0.026/297) #Kelvin to Volts, use 297; could be different locations for different sheets
Vipa = np.array(df.Vipa_Vs, dtype = 'float64') #is in volts*s
Vipb = np.array(df.Vipb_Vs, dtype = 'float64') #is in volts*s
Qa = np.array(df.Qa_As, dtype = 'float64') #As
Qb = np.array(df.Qb_As, dtype = 'float64') #As
#E_abs = DBD.get_plasma_E() #Joules, double check
E_abs = np.array(df.E_abs, dtype = 'float64') #unclear? Joules?
t_a_vec = np.array(df.ta_s, dtype = 'float64') #seconds
t_b_vec = np.array(df.tb_s, dtype = 'float64') #seconds
d_bead_vec = np.array(df.packing_size, dtype = 'float64')/1000 #mm --> m
h_rxtor_vec = np.array(df.height, dtype = 'float64')/100  #cm --> m
signature = df.signature #name



index_max = flowrate.shape[0]

In [7]:
E_abs

array([5.60860754e-07, 6.71729109e-07, 3.17600069e-07, 3.99160093e-07,
       4.45465394e-07, 5.28241322e-07, 5.05933233e-07, 5.66378637e-07,
       5.12705418e-07, 5.42710057e-07, 4.52867394e-07, 7.52942058e-07,
       6.37637116e-07, 4.66442839e-07, 4.19451270e-07, 3.26278519e-07,
       9.77563822e-07, 7.77183327e-07, 1.17187434e-06, 9.17556149e-07,
       1.21250351e-06, 7.27786474e-07, 7.98922519e-07, 8.08908098e-07])

In [8]:
Te

array([0.12115825, 0.13052525, 0.13954209, 0.13709091, 0.17061953,
       0.1493468 , 0.14908418, 0.18900337, 0.19285522, 0.16983165,
       0.1383165 , 0.12212121, 0.13857912, 0.16344108, 0.17683502,
       0.32381818, 0.21762963, 0.15232323, 0.1316633 , 0.12369697,
       0.12369697, 0.15652525, 0.13534007, 0.13490236])

In [9]:
#Now to do some solving of current conservation equations
array = []
for index in range(0,index_max):

    #Need:
    Q_a = float(Qa[index])
    Q_b = float(Qb[index])
    T_e = Te[index]
    #t_a = float(time_col[index,0]) #seconds
    #t_b = float(time_col[index,1]) #seconds
    t_a = float(t_a_vec[index])
    t_b = float(t_b_vec[index])
    t_c = float(1/freq[index]-t_a-t_b)
    t_b = t_c+t_b
    V_p_ta = float(Vipa[index]/t_a) #volts
    V_p_tb = float(Vipb[index]/(t_b+t_c))  #volts
    V_p_tc = 0.0
    Flow = flowrate[index]*6*(10**-5)-0.0001 #convert from % to m^3/s, see exccel "flow_calibration..."
    E = float(E_abs[index])
    ns = width_ns[index]
    temp_C_gas = temp_C[index]
    frequency = freq[index]
    volt = voltage[index]
    sign = signature[index]
    print(E)


    #Constants:
    d= 2.25*2.54*0.01 # m, 2.25" electrode diameter converted to cm (2.25) then to meters (0.01)
    A_a = (d**2)*3.14/4 #m^2 2.25" diameter converted to meters
    h = 0.75*2.54*0.01 #inner volume thickness, 0.75 inches to meters
    e_c = 1.6022*(10**-19) #elementary charge C = A*s
    m_e = 9.1095*(10**-31) # electron mass kg
    M_He = 4*1.661*(10**-27) #4 daltons, each Dalton = 1.661*10^-27 kg
    u_B = (e_c*T_e/M_He)**0.5  #bohm velocity, m/s C*V/kg = J/kg = kg⋅m2⋅s−2/kg = m2_s2 -sqrt - m_s
    f_ab = 1/((1+2*0.3)**2)  #square mesh, square hole side length: 0.001m (1mm), wire diameter: 0.3mm or 0.0003m.
    A_b = float(f_ab*A_a)   # m^2, assume only reactor-facing area. Plasma bathes this thing, should we use all area?
    v_e = ((8*e_c*T_e)/(np.pi*m_e))**0.5 #m/s
    Tg = 300 #kelvin 
    T_g = 300 #kelvin
    ng = ((1/22.4)*294/272)*(6.022*10**23)*1000   #mole/L*(T/Tideal)*(atoms/mole)*(1000L/m^3) --> atoms/m^3; invert the temp!
    n_g = ng
    
    d_bead = d_bead_vec[index]
    #A_bead = ((d_bead*10**-3)**2)*(4)*np.pi*(1/4) #m^2, 2.3mm diam
    A_bead = ((d_bead)**2)*(4)*np.pi*(1/4) #m^2, 2.3mm diam, new
    A_bead = float(A_bead)
    #V_bead =  ((d_bead*10**-3)**3)*(4/3)*np.pi*(1/8) # m^3 
    V_bead =  ((d_bead)**3)*(4/3)*np.pi*(1/8) # m^3 new!
    
    #mass_bead = 107.24 #replaced this too!
    if h_rxtor_vec[index] ==0.02:
        mass_bead = 107.24
    elif h_rxtor_vec[index] ==0.03:
        mass_bead = 168
    elif h_rxtor_vec[index] ==0.05:
        mass_bead = 238.2
    else:
        print('problem with reactor to beads')
    
    #h = h_rxtor_vec[index]/100 #cm --> m  #h is repeated?  #uncomment in the future
    h = h_rxtor_vec[index] # m  #h is repeated, this is the new value
    Volume_rxtor = float(A_a*h) # m^3 
    V_all_beads = (mass_bead*16/0.45)*V_bead #m^3
    
    
    #A_tot = np.pi*d*h+(107.24*16/0.45)*A_bead  #m^2
    A_tot = np.pi*d*h+(mass_bead*16/0.45)*A_bead  #m^2
    A_tot = float(A_tot)                       #new!
    Volume = float(Volume_rxtor-V_all_beads)         #new!

    
    ev_to_j = 1.602*10**-19

    eps_0 = 8.854*(10**-12) #F/m = [J/V^2]*[m^-1] =  [kg m^2 /s^2]*[V^-2]*[m^-1] = [kg m/s^2]*[V^-2]
    kappa = 4.6 #dimensionless
    epsilon = eps_0*kappa #Dielectric constant
    
    k_b = 1.38*(10**-23) #m2kg_s2_K


    def T_e_basis(T_e):
        u_B = (e_c*T_e/M_He)**0.5
        v_e = ((8*e_c*T_e)/(3.14159*m_e))**0.5
        K_elastic = 4.937*(10**-14)*(T_e**0.2579) #R1, elastic
        E_elastic = 3*m_e*T_e/M_He

        K_iz =  1.5942*(10**-14)*(T_e**1.6767)* exp(-49.2949/T_e) #R3, ionization
        E_iz = 24.59 #eV #R3

        K_iz_exc= 2.254*(10**-13)*(T_e**-0.1241)*exp(-5.725/T_e) #R4, ionization of excied species 
        E_iz_exc = 4.77 #eV E_{iz-exc} = Ionization - first excitation = 24.59- 19.82

        K_exc = 1.0022*(10**-13)*(T_e**-0.96989)*exp(-33.8044/T_e)+3.8066*(10**-15)*(T_e**0.74604)*exp(-32.1884/T_e)
        E_exc = 19.82 #eV

        K_2_iz = 2.03*(10**-15)*(T_g/300)**0.5 #R110, no delta E (costs no electron energy, gas temperature assumed constant, excitation modes already assumed to be lost).

        K_loss = 2*(10**-46) #R122, just loss of He*

        my_tuple = [u_B, v_e, K_elastic, K_iz, K_iz_exc, K_exc, K_2_iz, K_loss]
        return my_tuple


    def f_n_He_exc(K_2_iz,K_loss, K_iz_exc, K_exc, n_e, n_g):
        a = -2*K_2_iz
        b = -K_loss*(n_g**2)-K_iz_exc*n_e
        c = K_exc*n_e*n_g
        n_He_exc = (-b-(b**2-4*a*c)**0.5)/(2*a)
        return n_He_exc

    #TWO TEMPERATURE CASE!

    #x0 = n_sa
    #x1 = n_sb
    #x2 = n_e
    #x3 = T_e

    #testing the mass balance

    def create_model_high_Te_kinetics():
            m = ConcreteModel()

            m.set2 = Set(initialize=list(range(2)))
            m.x = Var(m.set2, initialize=x_guess)


            def x_energy(m):

                T_e = Te[index]

                my_tup = T_e_basis(T_e)
                u_B = my_tup[0]
                v_e = my_tup[1]
                K_elastic = my_tup[2]
                K_iz = my_tup[3]
                K_iz_exc = my_tup[4]
                K_exc = my_tup[5]
                K_2_iz = my_tup[6]
                K_loss = my_tup[7]

                n_He_exc = f_n_He_exc(K_2_iz,K_loss, K_iz_exc, K_exc, m.x[0],n_g)


                #print(Q_b)
                n_sa = Q_b/(e_c*A_a*t_b*(u_B))  #prev: negative, and uB-0.025*v_e
                #print(n_sa/10**15)
                n_sb = -Q_b/(e_c*A_b*t_b*(u_B-0.25*v_e)) #prev: positive, and uB
                #print(n_sb/10**15)
                #n_sa = Q_b/(e_c*A_a*(t_b+t_c)*u_B)
                #print(n_sa/10**15)
                #n_sb = -Q_b/(e_c*A_b*(t_b+t_c)*(u_B-0.25*v_e))
                #print(n_sb/10**15)


                ####
                T_e_a = m.x[1]*Te[index]

                my_tup_a = T_e_basis(T_e_a)
                u_B_a = my_tup_a[0]
                v_e_a = my_tup_a[1]
                K_elastic_a = my_tup_a[2]
                K_iz_a = my_tup_a[3]
                K_iz_exc_a = my_tup_a[4]
                K_exc_a = my_tup_a[5]
                K_2_iz_a = my_tup_a[6]
                K_loss_a = my_tup_a[7]

                n_He_exc_a = f_n_He_exc(K_2_iz_a,K_loss_a, K_iz_exc_a, K_exc_a, m.x[0],n_g)

                E_elastic = e_c*3*m_e*T_e/M_He
                E_elastic_a = e_c*3*m_e*T_e_a/M_He
                E_iz = 24.59*ev_to_j #eV #R3
                E_iz_exc = 4.77*ev_to_j #eV E_{iz-exc} = Ionization - first excitation = 24.59- 19.82
                E_exc = 19.82*ev_to_j #eV


                #calculate energy during the off pulse first

                #V_p_tb is negative. It occurs at electrode A during time b. Put a negative sign in front of it.
                rxn_energy_tb = m.x[0]*n_g*Volume*(K_exc*E_exc+K_elastic*E_elastic+K_iz*E_iz)*t_b + m.x[0]*n_He_exc*Volume*K_iz_exc*E_iz_exc*t_b
                wall_loss_time_b = n_sa*u_B*A_a*t_b*e_c*(0.5*T_e+V_p_tb) + n_sb*u_B*A_b*t_b*e_c*(0.5*T_e) - 0.25*n_sb*A_b*e_c*(-2*T_e)*t_b*v_e
                floating_loss_time_b = m.x[0]*exp(-1/2)*u_B*e_c*(5/2)*T_e*A_tot*t_b

                #calculate energy during the on pulse first

                rxn_energy_ta = m.x[0]*n_g*Volume*(K_exc_a*E_exc+K_elastic_a*E_elastic_a+K_iz_a*E_iz)*t_a + m.x[0]*n_He_exc_a*Volume*K_iz_exc_a*E_iz_exc*t_a

                wall_loss_time_a = n_sa*u_B_a*A_a*t_a*e_c*(0.5*T_e_a) - 0.25*n_sa*A_a*e_c*(-2*T_e_a)*t_a*v_e_a + n_sb*u_B_a*A_b*t_a*e_c*(0.5*T_e_a+V_p_ta)
                floating_loss_time_a = m.x[0]*exp(-1/2)*u_B_a*e_c*(5/2)*T_e_a*A_tot*t_a


                rxn_energy = rxn_energy_tb+rxn_energy_ta
                wall_loss = wall_loss_time_b+wall_loss_time_a
                floating_loss = floating_loss_time_b+floating_loss_time_a



                return 0== -1+rxn_energy/E+wall_loss/E+floating_loss/E

            def x_mass(m):

                T_e = Te[index]

                my_tup = T_e_basis(T_e)
                u_B = my_tup[0]
                v_e = my_tup[1]
                K_elastic = my_tup[2]
                K_iz = my_tup[3]
                K_iz_exc = my_tup[4]
                K_exc = my_tup[5]
                K_2_iz = my_tup[6]
                K_loss = my_tup[7]

                n_He_exc = f_n_He_exc(K_2_iz,K_loss, K_iz_exc, K_exc, m.x[0],n_g)


                #print(Q_b)
                n_sa = -Q_b/(e_c*A_a*t_b*(u_B-0.25*v_e)) #switch back if problems
                #print(n_sa/10**15)
                n_sb = Q_b/(e_c*A_b*t_b*u_B)
                #print(n_sb/10**15)
                #n_sa = Q_b/(e_c*A_a*(t_b+t_c)*u_B)
                #print(n_sa/10**15)
                #n_sb = -Q_b/(e_c*A_b*(t_b+t_c)*(u_B-0.25*v_e))
                #print(n_sb/10**15)


                ####
                T_e_a = m.x[1]*Te[index]

                my_tup_a = T_e_basis(T_e_a)
                u_B_a = my_tup_a[0]
                v_e_a = my_tup_a[1]
                K_elastic_a = my_tup_a[2]
                K_iz_a = my_tup_a[3]
                K_iz_exc_a = my_tup_a[4]
                K_exc_a = my_tup_a[5]
                K_2_iz_a = my_tup_a[6]
                K_loss_a = my_tup_a[7]

                n_He_exc_a = f_n_He_exc(K_2_iz_a,K_loss_a, K_iz_exc_a, K_exc_a, m.x[0],n_g)


                #0 = Volume*n_e*n_g*K_iz*(t_a+t_b)+Volume*n_He_exc*n_e*K_2_iz*(t_a+t_b)-n_sa*u_B*A_a*(t_a+t_b)-n_sb*u_B*A_b*(t_a+t_b)-n_sp*u_B*A_tot*(t_a+t_b)
                #0 = Volume*m.x[2]*n_g*K_iz*(t_a+t_b)+Volume*n_He_exc*n_e*K_2_iz*(t_a+t_b)-m.x[0]*u_B*A_a*(t_a+t_b)-m.x[1]*u_B*A_b*(t_a+t_b)-m.x[3]*np.exp(-1/2)*u_B*A_tot*(t_a+t_b)
                packing_loss = m.x[0]*exp(-1/2)*(u_B_a*t_a+u_B*t_b)*A_tot
                return 0==Volume*m.x[0]*n_g*(K_iz_a*t_a+K_iz*t_b)+Volume*(n_He_exc_a*t_a*K_iz_exc_a+n_He_exc*t_b*K_iz_exc)*m.x[0]+Volume*((n_He_exc_a**2)*t_a*K_2_iz_a+(n_He_exc**2)*t_b*K_2_iz)-n_sa*A_a*(u_B_a*t_a+u_B*t_b)-n_sb*A_b*(u_B_a*t_a+u_B*t_b)-packing_loss
                #33-35, 12-14, no wall loss



            m.con1 = Constraint(rule=x_energy)
            m.con2 = Constraint(rule=x_mass)
            return m
    guess_n_e = 10**11
    guess_T = 20
    conv = False
    
    #x_guess = {0:10**18, 1: 35}
    #mod = create_model_high_Te_kinetics()
    
    while not(conv) and (guess_n_e<=10**14):
        
        guess_T = 20
        
        while not(conv) and (guess_T<=72):  #previously it was 40 or so
   

        #solver = SolverFactory('ipopt')
            try:
                x_guess = {0:guess_n_e, 1: guess_T}
                mod = create_model_high_Te_kinetics()
                solver=get_solver()
                solver.options['linear_solver'] = 'mumps' # if you don't have ma57, change to ma27 or comment this line
                solver.options['halt_on_ampl_error'] = 'yes'
                solver.options['max_iter'] = 10000 # change this as needed
                solver.options['tol'] = 1E-6 # change this as needed

                result = solver.solve(mod, tee=True)


                #outputs:
                print([value(mod.x[_]) for _ in range(2)])

                n_e = value(mod.x[0])
                T_e_multiplier = value(mod.x[1])
                conv = True

            except:
                if guess_n_e == 10**14 and guess_T ==72:
                    n_e = 1
                    T_e_multiplier = 0.001
                    conv = True

#issue: stopping at 10**18 and T=36 in both if and the while means the if is never triggered
                else:
                    guess_T = guess_T+4
            
        guess_n_e = guess_n_e*10

    T_e = Te[index]


    my_tup = T_e_basis(T_e)
    u_B = my_tup[0]
    v_e = my_tup[1]
    K_elastic = my_tup[2]
    K_iz = my_tup[3]
    K_iz_exc = my_tup[4]
    K_exc = my_tup[5]
    K_2_iz = my_tup[6]
    K_loss = my_tup[7]

    n_sa = Q_b/(e_c*A_a*t_b*(u_B))
                #print(n_sa/10**15)
    n_sb = -Q_b/(e_c*A_b*t_b*(u_B-0.25*v_e))

    n_He_exc = f_n_He_exc(K_2_iz,K_loss, K_iz_exc, K_exc, n_e,n_g)    

    
    
        
    if n_sb < 0:
        T_e_multiplier = 0.001
        n_e = 1
    T_e_a = T_e*T_e_multiplier


    my_tup_a = T_e_basis(T_e_a)
    u_B_a = my_tup_a[0]
    v_e_a = my_tup_a[1]
    K_elastic_a = my_tup_a[2]
    K_iz_a = my_tup_a[3]
    K_iz_exc_a = my_tup_a[4]
    K_exc_a = my_tup_a[5]
    K_2_iz_a = my_tup_a[6]
    K_loss_a = my_tup_a[7]

    n_He_exc_a = f_n_He_exc(K_2_iz_a,K_loss_a, K_iz_exc_a, K_exc_a, n_e,n_g)

    E_elastic = e_c*3*m_e*T_e/M_He
    E_elastic_a = e_c*3*m_e*T_e_a/M_He
    E_iz = 24.59*ev_to_j #eV #R3
    E_iz_exc = 4.77*ev_to_j #eV E_{iz-exc} = Ionization - first excitation = 24.59- 19.82
    E_exc = 19.82*ev_to_j #eV


    # MB Terms
    ta_He_vol_ion = Volume*n_e*n_g*K_iz_a*t_a
    ta_He_vol_ion_name = 'ta Vol He Ionization'
    
    tb_He_vol_ion = Volume*n_e*n_g*K_iz*t_b
    tb_He_vol_ion_name = 'tb Vol He Ionization'
    
    ta_vol_He_exc_ion = Volume*n_e*n_He_exc_a*K_iz_exc_a*t_a
    ta_vol_He_exc_ion_name = 'ta Vol He_exc Ionization'
    
    tb_vol_He_exc_ion = Volume*n_e*n_He_exc*K_iz_exc*t_b
    tb_vol_He_exc_ion_name = 'tb Vol He_exc Ionization'
    
    ta_vol_double_He_exc_ion = Volume*(n_He_exc_a**2)*K_2_iz_a*t_a
    ta_vol_double_He_exc_ion_name = 'ta Vol double He_exc Ionization'
    
    tb_vol_double_He_exc_ion = Volume*(n_He_exc**2)*K_2_iz*t_b
    tb_vol_double_He_exc_ion_name = 'tb Vol double He_exc Ionization'
    
    ta_loss_electrode_a = A_a*n_sa*u_B_a*t_a
    ta_loss_electrode_a_name = 'ta loss to electrode a'
    
    tb_loss_electrode_a = A_a*n_sa*u_B*t_b
    tb_loss_electrode_a_name = 'tb loss to electrode a'
    
    ta_loss_electrode_b = A_b*n_sb*u_B_a*t_a
    ta_loss_electrode_b_name = 'ta loss electrode b'
    
    tb_loss_electrode_b = A_b*n_sb*u_B*t_b
    tb_loss_electrode_b_name = 'tb loss electrode b'
    
    ta_packing_wall_loss = n_e*exp(-1/2)*u_B_a*A_tot*t_a
    ta_packing_wall_loss_name = 'ta_packing_wall_loss'
    
    tb_packing_wall_loss = n_e*exp(-1/2)*u_B*A_tot*t_b
    tb_packing_wall_loss_name = 'tb_packing_wall_loss'
    
    mass_balance_rates = np.array( [ta_He_vol_ion, tb_He_vol_ion, ta_vol_He_exc_ion, tb_vol_He_exc_ion, ta_vol_double_He_exc_ion, tb_vol_double_He_exc_ion, ta_loss_electrode_a, tb_loss_electrode_a, ta_loss_electrode_b, tb_loss_electrode_b, ta_packing_wall_loss, tb_packing_wall_loss ] )
    mass_balance_rates_names = [ta_He_vol_ion_name,
                               tb_He_vol_ion_name,
                               ta_vol_He_exc_ion_name,
                               tb_vol_He_exc_ion_name,
                               ta_vol_double_He_exc_ion_name,
                               tb_vol_double_He_exc_ion_name,
                                ta_loss_electrode_a_name,
                                tb_loss_electrode_a_name,
                                ta_loss_electrode_b_name,
                                tb_loss_electrode_b_name,
                                ta_packing_wall_loss_name,
                                tb_packing_wall_loss_name
                               ]
    
    #EB Terms
#rxn_energy_tb = m.x[0]*n_g*Volume*(K_exc*E_exc+K_elastic*E_elastic+K_iz*E_iz)*t_b + m.x[0]*n_He_exc*Volume*K_iz_exc*E_iz_exc*t_b
#wall_loss_time_b = n_sa*u_B*A_a*t_b*e_c*(0.5*T_e-V_p_tb) + n_sb*u_B*A_b*t_b*e_c*(0.5*T_e) - 0.25*n_sb*A_b*e_c*(-2*T_e)*t_b*v_e
#floating_loss_time_b = m.x[0]*exp(-1/2)*u_B*e_c*(5/2)*T_e*A_tot*t_b
    
#rxn_energy_ta = m.x[0]*n_g*Volume*(K_exc_a*E_exc+K_elastic_a*E_elastic_a+K_iz_a*E_iz)*t_a + m.x[0]*n_He_exc_a*Volume*K_iz_exc_a*E_iz_exc*t_a
#wall_loss_time_a = n_sa*u_B_a*A_a*t_a*e_c*(0.5*T_e_a) - 0.25*n_sa*A_a*e_c*(-2*T_e_a)*t_a*v_e_a + n_sb*u_B_a*A_b*t_a*e_c*(0.5*T_e_a+V_p_ta)
#floating_loss_time_a = m.x[0]*exp(-1/2)*u_B_a*e_c*(5/2)*T_e_a*A_tot*t_a    
    
    # first term is just E.
    tb_excitation = n_e*n_g*Volume*K_exc*E_exc*t_b
    tb_excitation_name = 'tb_excitation'
    
    tb_elastic = n_e*n_g*Volume*K_elastic*E_elastic*t_b
    tb_elastic_name = 'tb_elastic'
    
    tb_ionization = n_e*n_g*Volume*K_iz*E_iz*t_b
    tb_ionization_name = 'tb_ionization'
    
    tb_exc_ionization = n_e*n_He_exc*K_iz_exc*E_iz_exc*t_b
    tb_exc_ionization_name = 'tb_exc_ionization'
    
    tb_ion_energy_loss_electrode_a = n_sa*u_B*A_a*t_b*e_c*(0.5*T_e+V_p_tb)
    tb_ion_energy_loss_electrode_a_name = 'tb_ion_energy_loss_electrode_a'
    
    tb_ion_energy_loss_electrode_b =  n_sb*u_B*A_b*t_b*e_c*(0.5*T_e)
    tb_ion_energy_loss_electrode_b_name = 'tb_ion_energy_loss_electrode_b'
    
    tb_electron_energy_loss_electrode_b = -1*0.25*n_sb*A_b*e_c*(-2*T_e)*t_b*v_e
    tb_electron_energy_loss_electrode_b_name = 'tb_electron_energy_loss_electrode_b'
    
    tb_floating_loss = n_e*exp(-1/2)*u_B*e_c*(5/2)*T_e*A_tot*t_b
    tb_floating_loss_name = 'tb_floating_loss'
    
    
    ta_excitation = n_e*n_g*Volume*K_exc_a*E_exc*t_a
    ta_excitation_name = 'ta_excitation'
    
    ta_elastic = n_e*n_g*Volume*K_elastic_a*E_elastic_a*t_a
    ta_elastic_name = 'ta_elastic'
    
    ta_ionization = n_e*n_g*Volume*K_iz_a*E_iz*t_a
    ta_ionization_name = 'ta_ionization'
    
    ta_exc_ionization = n_e*n_He_exc_a*Volume*K_iz_exc_a*E_iz_exc*t_a
    ta_exc_ionization_name = 'ta_exc_ionization'
    
    ta_ion_energy_loss_electrode_a = n_sa*u_B_a*A_a*t_a*e_c*(0.5*T_e_a)
    ta_ion_energy_loss_electrode_a_name = 'ta_ion_energy_loss_electrode_a'
    
    ta_electron_energy_loss_electrode_a = -0.25*n_sa*A_a*e_c*(-2*T_e_a)*t_a*v_e_a
    ta_electron_energy_loss_electrode_a_name = 'ta_electron_energy_loss_electrode_a'
    
    ta_ion_energy_loss_electrode_b = n_sb*u_B_a*A_b*t_a*e_c*(0.5*T_e_a+V_p_ta)
    ta_ion_energy_loss_electrode_b_name = 'ta_ion_energy_loss_electrode_b'
    
    ta_floating_energy_loss = n_e*exp(-1/2)*u_B_a*e_c*(5/2)*T_e_a*A_tot*t_a
    ta_floating_energy_loss_name = 'ta_floating_energy_loss'
    
    Energy_in_name = 'Energy_in'
    
    energy_balance_rates = np.array([E,
                                     tb_excitation,
                                    tb_elastic,
                                    tb_ionization,
                                    tb_exc_ionization,
                                    tb_ion_energy_loss_electrode_a,
                                    tb_ion_energy_loss_electrode_b,
                                    tb_electron_energy_loss_electrode_b,
                                    tb_floating_loss,
                                    ta_excitation,
                                    ta_elastic,
                                    ta_ionization,
                                    ta_exc_ionization,
                                    ta_ion_energy_loss_electrode_a,
                                    ta_electron_energy_loss_electrode_a,
                                    ta_ion_energy_loss_electrode_b,
                                    ta_floating_energy_loss
                                    ])
    energy_balance_rates_names =  [Energy_in_name,
                                    tb_excitation_name,
                                    tb_elastic_name,
                                    tb_ionization_name,
                                    tb_exc_ionization_name,
                                    tb_ion_energy_loss_electrode_a_name,
                                    tb_ion_energy_loss_electrode_b_name,
                                    tb_electron_energy_loss_electrode_b_name,
                                    tb_floating_loss_name,
                                    ta_excitation_name,
                                    ta_elastic_name,
                                    ta_ionization_name,
                                    ta_exc_ionization_name,
                                    ta_ion_energy_loss_electrode_a_name,
                                    ta_electron_energy_loss_electrode_a_name,
                                    ta_ion_energy_loss_electrode_b_name,
                                    ta_floating_energy_loss_name
                                    ]
    
    

#1. electron density (n_e)
# 2. Boh velocity--off period (u_B)
# 3. Bohm velocity--pulse (u_B_a)
# 4. total area including packing (A_tot)
# 5. pulse time t_a
# 6. off pulse time t_b
# 7. Free volume (Volume)
# 8. helium gas density (n_g)
# 9. K_ioniation during the pulse (K_iz_a)
# 10. K_ionization off pulse (K_iz)
# 11. K_double_impact_excitation during pulse (K_2_iz_a)
# 12. K_double_impact_excitation off pulse (K_2_iz)
# 12. K_excitation_ionization during pulse (K_iz_exc_a)
# 13. K_excitation_ionization off pulse (K_iz_exc)
# 14. electrode sheath concentration (n_sa)
# 15. electrode b sheath concentration (n_sb)
# 16. electrode A area (A_a)
# 17. electrode B area (A_b)
# Parameters that can be included:
# reactor height, h
# permitivvity of the particles, epsilon
# Flowrate (Flow)


# rxn_energy_tb = m.x[0]*n_g*Volume*(K_exc*E_exc+K_elastic*E_elastic+K_iz*E_iz)*t_b + m.x[0]*n_He_exc*Volume*K_iz_exc*E_iz_exc*t_b
# wall_loss_time_b = n_sa*u_B*A_a*t_b*e_c*(0.5*T_e-V_p_tb) + n_sb*u_B*A_b*t_b*e_c*(0.5*T_e) - 0.25*n_sb*A_b*e_c*(-2*T_e)*t_b*v_e
# floating_loss_time_b = m.x[0]*exp(-1/2)*u_B*e_c*(5/2)*T_e*A_tot*t_b
# rxn_energy_ta = m.x[0]*n_g*Volume*(K_exc_a*E_exc+K_elastic_a*E_elastic_a+K_iz_a*E_iz)*t_a + m.x[0]*n_He_exc_a*Volume*K_iz_exc_a*E_iz_exc*t_a
# wall_loss_time_a = n_sa*u_B_a*A_a*t_a*e_c*(0.5*T_e_a) - 0.25*n_sa*A_a*e_c*(-2*T_e_a)*t_a*v_e_a + n_sb*u_B_a*A_b*t_a*e_c*(0.5*T_e_a+V_p_ta)
# floating_loss_time_a = m.x[0]*exp(-1/2)*u_B_a*e_c*(5/2)*T_e_a*A_tot*t_a

    #need to calculate or retrieve:
    storage_i_a = np.array([ t_a, Q_a, V_p_ta,T_e_a, n_He_exc_a, u_B_a, v_e_a, K_2_iz_a,K_loss_a, K_iz_exc_a, K_exc_a, K_iz_a, K_elastic_a, E_elastic_a ])
    storage_i_b = np.array([ t_b, Q_b, V_p_tb,T_e, n_He_exc, u_B, v_e, K_2_iz,K_loss, K_iz_exc, K_exc, K_iz, K_elastic, E_elastic ] )
    storage_all = np.array([E, n_sa,n_sb, n_e, n_g, T_g, E_iz, E_iz_exc, E_exc, e_c, m_e, M_He,  epsilon, eps_0, k_b ])
    storage_geom = np.array([Volume_rxtor, V_all_beads, A_a,A_b, h, Volume, A_bead, A_tot, frequency, Flow, temp_C_gas, volt, ns , d_bead ])
    storage_sign = sign
    mass_balance = np.array([n_e, n_g, u_B, A_tot, t_a, t_b, Volume, K_iz_a, K_2_iz_a, K_iz_exc_a, n_sa, n_sb, A_a, A_b, n_He_exc_a])
    energy_balance = np.array([T_e, T_g, Volume, n_e, n_g, A_a, A_b, e_c, A_tot, E, K_exc_a*E_exc, K_iz_a*E_iz, K_iz_exc_a*E_iz_exc, K_elastic*E_elastic, t_a, n_sa, n_sb, V_p_ta, t_b, u_B, V_p_tb, v_e, n_He_exc_a  ])
    #mass_balance_rates are stored above!
    #energy_balance_rates are stored above!
    
    if index == 0:
        
        array_a = storage_i_a
        array_b = storage_i_b
        array_all = storage_all
        array_geom = storage_geom
        vec_sign = []
        vec_sign.append(storage_sign)
        array_mb = mass_balance
        array_eb = energy_balance
        array_mb_rates = mass_balance_rates
        array_eb_rates = energy_balance_rates
    else:
        array_a = np.vstack((array_a,storage_i_a))
        array_b = np.vstack((array_b,storage_i_b))
        array_all = np.vstack((array_all,storage_all))
        array_geom = np.vstack((array_geom,storage_geom))
        vec_sign.append(storage_sign)
        array_mb = np.vstack((array_mb, mass_balance))
        array_eb = np.vstack((array_eb, energy_balance))
        array_mb_rates = np.vstack((array_mb_rates, mass_balance_rates))
        array_eb_rates = np.vstack((array_eb_rates, energy_balance_rates))

5.608607535190455e-07
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=10000
linear_solver=mumps
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the follo

ERROR: Solver (ipopt) returned non-zero return code (1)
ERROR: See the solver log above for diagnostic information.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=10000
linear_solver=mumps
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    

   3  0.0000000e+00 6.34e+09 0.00e+00  -1.0 1.90e+12    -  1.00e+00 1.25e-01h  4
   4  0.0000000e+00 5.92e+09 0.00e+00  -1.0 1.66e+12    -  1.00e+00 1.00e+00h  1
   5  0.0000000e+00 3.05e+08 0.00e+00  -1.0 1.25e+09    -  1.00e+00 1.00e+00h  1
   6  0.0000000e+00 5.97e+05 0.00e+00  -1.0 2.72e+09    -  1.00e+00 1.00e+00h  1
   7  0.0000000e+00 1.40e+00 0.00e+00  -2.5 8.74e+06    -  1.00e+00 1.00e+00h  1
   8  0.0000000e+00 1.04e-04 0.00e+00  -7.0 3.49e+01    -  1.00e+00 1.00e+00h  1
   9  0.0000000e+00 5.25e-05 0.00e+00  -7.0 9.45e-04    -  1.00e+00 1.00e+00   0

Number of Iterations....: 9

                                   (scaled)                 (unscaled)
Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
Dual infeasibility......:   0.0000000000000000e+00    0.0000000000000000e+00
Constraint violation....:   9.5365657772579842e-12    5.2452087402343750e-05
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......

ERROR: Solver (ipopt) returned non-zero return code (1)
ERROR: See the solver log above for diagnostic information.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=10000
linear_solver=mumps
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    

ERROR: Solver (ipopt) returned non-zero return code (1)
ERROR: See the solver log above for diagnostic information.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=10000
linear_solver=mumps
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    

...

[382698903287.6135, 47.018019557394446]
7.989225187027366e-07
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=10000
linear_solver=mumps
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes

ERROR: Solver (ipopt) returned non-zero return code (1)
ERROR: See the solver log above for diagnostic information.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=10000
linear_solver=mumps
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    

   2  0.0000000e+00 1.13e+09 0.00e+00  -1.0 9.60e+11    -  1.00e+00 5.00e-01h  2
   3  0.0000000e+00 8.30e+08 0.00e+00  -1.0 4.80e+11    -  1.00e+00 1.00e+00H  1
   4  0.0000000e+00 3.84e+06 0.00e+00  -1.0 6.11e+07    -  1.00e+00 1.00e+00h  1
   5  0.0000000e+00 5.98e+01 0.00e+00  -1.7 6.86e+06    -  1.00e+00 1.00e+00h  1
   6  0.0000000e+00 8.30e-05 0.00e+00  -7.0 1.49e+02    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 6

                                   (scaled)                 (unscaled)
Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
Dual infeasibility......:   0.0000000000000000e+00    0.0000000000000000e+00
Constraint violation....:   6.4925117106915921e-12    8.2969665527343750e-05
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   6.4925117106915921e-12    8.2969665527343750e-05


Number of objective function evaluations             = 15
Number of objective gradient evaluations         

ERROR: Solver (ipopt) returned non-zero return code (1)
ERROR: See the solver log above for diagnostic information.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=10000
linear_solver=mumps
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    


Number of Iterations....: 7

                                   (scaled)                 (unscaled)
Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
Dual infeasibility......:   0.0000000000000000e+00    0.0000000000000000e+00
Constraint violation....:   9.9946380631620410e-12    6.4849853515625000e-05
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   9.9946380631620410e-12    6.4849853515625000e-05


Number of objective function evaluations             = 15
Number of objective gradient evaluations             = 8
Number of equality constraint evaluations            = 15
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 8
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 7
Total CPU secs in IPOPT (w/o function evaluations)   =      0.003
Total CPU secs in NLP function evaluations   

In [10]:
idx = np.arange(index_max)
print(idx[array_all[:,3]<5])
bad_index = idx[array_all[:,3]<5]
good_index = np.asarray(idx[array_all[:,3]>5], dtype = int)
print(np.where(array_all[:,3]<5)[0].shape)
print(np.where(array_all[:,3]<5)[0])

[]
(0,)
[]


In [11]:
#time_a_data

#units: m s atom kg Amp K


df1_names = [ 't_a_seconds', 'Q_a_As', 'V_p_ta_kgm2_s3_A','T_e_a_kgm2_s3_A', 'n_He_exc_a_atoms_m3', 'u_B_a_m_s', 'v_e_a_m_s', 'K_2_iz_a_m3_s_atom','K_loss_a_m6_s_atom2', 'K_iz_exc_a_m3_s_atom', 'K_exc_a_m3_s_atom', 'K_iz_a_m3_s_atom', 'K_elastic_a_m3_s_atom', 'E_elastic_a_kgm2_s2' ]
D_df_1 = np.array(
    [    #t_a_s  #Q_a_As V_p_ta_kgm2_s3_A   T_e_a_kgm2_s3_A     n_He_exc_a_atoms_m3       u_B_a_m_s     v_e_a_m_s    K_2_iz_a_m3_s_atom     K_loss_a_m6_s_atom2   K_iz_exc_a_m3_s_atom     K_exc_a_m3_s_atom   K_iz_a_m3_s_atom      K_elastic_a_m3_s_atom      E_elastic_a_kgm2_s2
        [ 0.,       0.,        2.,                 2.,                   -3.,                1.,           1.,                3.,                    6.,                    3.,                   3.,                   3.,                   3.,                         2 ], # m
        [ 1.,       1.,       -3.,                 -3.,                   0.,                -1.,          -1.,               -1.,                   -1.,                    -1.,                 -1.,                  -1.,                   -1.,                         -2 ], # s
        [ 0.,       0.,        0.,                0.,                   1.,                 0.,           0.,               -1.,                   -2.,                    -1.,                 -1.,                  -1.,                   -1.,                         0 ], # atom
        [ 0.,       0.,        1.,                 1.,                   0.,                 0.,           0.,                0.,                    0.,                    0.,                   0.,                   0.,                    0.,                          1], # kg
        [ 0.,       1.,       -1.,                -1.,                   0.,                 0.,           0.,                0.,                    0.,                    0.,                   0.,                   0.,                    0.,                          0], # Ampere
        [ 0.,       0.,        0.,                 0.,                   0.,                 0.,           0.,                0.,                    0.,                    0.,                   0.,                   0.,                    0.,                          0], # K
    ],
)

In [12]:
#time_b_data

#units: m s atom kg Amp K

df2_names =  [ 't_b_seconds', 'Q_b_As', 'V_p_tb_kgm2_s3_A','T_e_kgm2_s3_A', 'n_He_exc_atoms_m3', 'u_B_m_s', 'v_e_m_s','K_2_iz_m3_s_atom','K_loss_m6_s_atom2', 'K_iz_exc_m3_s_atom', 'K_exc_m3_s_atom', 'K_iz_m3_s_atom', 'K_elastic_m3_s_atom', 'E_elastic_kgm2_s2' ]
D_df_2 = np.array(
    [    #t_b_s  #Q_b_As V_p_tb_kgm2_s3_A   T_e_kgm2_s3_A      n_He_exc_atoms_m3          u_B_m_s       v_e_m_s      K_2_iz_m3_s_atom        K_loss_m6_s_atom2     K_iz_exc_m3_s_atom     K_exc_m3_s_atom   K_iz_m3_s_atom      K_elastic_m3_s_atom      E_elastic_kgm2_s2
        [ 0.,       0.,        2.,                 2.,                   -3.,                1.,           1.,                3.,                    6.,                    3.,                   3.,                  3.,                 3.,                         2 ], # m
        [ 1.,       1.,       -3.,                 -3.,                   0.,               -1.,          -1.,               -1.,                   -1.,                    -1.,                 -1.,                 -1.,                 -1.,                         -2 ], # s
        [ 0.,       0.,        0.,                 0.,                   1.,                 0.,           0.,               -1.,                   -2.,                    -1.,                 -1.,                 -1.,                 -1.,                         0 ], # atom
        [ 0.,       0.,        1.,                 1.,                   0.,                 0.,           0.,                0.,                    0.,                    0.,                   0.,                  0.,                  0.,                          1], # kg
        [ 0.,       1.,       -1.,                -1.,                   0.,                 0.,           0.,                0.,                    0.,                    0.,                   0.,                  0.,                  0.,                          0], # Ampere
        [ 0.,       0.,        0.,                 0.,                   0.,                 0.,           0.,                0.,                    0.,                    0.,                   0.,                  0.,                  0.,                          0], # K
    ],
)

In [13]:
#other data

#units: m s atom kg Amp K

df3_names = ['E_period_kgm2_s2', 'n_sa_atoms_m3','n_sb_atoms_m3', 'n_e_electrons_m3', 'n_g_atoms_m3', 'T_g_kelvin', 'E_iz_kgm2_s2', 'E_iz_exc_kgm2_s2', 'E_exc_kgm2_s2', 'e_c_As', 'm_e_kg', 'M_He_kg',  'epsilon_A2s4_kg_m3', 'eps_0_A2s4_kg_m3', 'k_b_kgm2_s2_K']
D_df_3 = np.array(
    [    #E_period_kgm2_s2  #n_sa_atoms_m3     n_sb_atoms_m3   n_e_electrons_m3      n_g_atoms_m3          T_g_kelvin      E_iz_kgm2_s2      E_iz_exc_kgm2_s2       E_exc_kgm2_s2     e_c_As     m_e_kg   M_He_kg      epsilon_A2s4_kg_m3      eps_0_A2s4_kg_m3     k_b_kgm2_s2_K
        [ 2.,                 -3.,                  -3.,             -3.,                  -3.,                 0.,           2.,                2.,                    2.,              0.,        0.,       0.,              -3.,                   -3,                2 ], # m
        [-2.,                  0.,                   0.,              0.,                   0.,                 0.,          -2.,               -2.,                   -2.,              1.,        0.,       0.,               4.,                   4,               -2 ], # s
        [ 0.,                  1.,                   1.,              1.,                   1.,                 0.,           0.,                0.,                    0.,              0.,        0.,       0.,               0.,                   0,                0 ], # atom
        [ 1.,                  0.,                   0.,              0.,                   0.,                 0.,           1.,                1.,                    1.,              0.,        1.,       1.,               -1.,                   -1,                1 ], # kg
        [ 0.,                  0.,                   0.,              0.,                   0.,                 0.,           0.,                0.,                    0.,              1.,        0.,       0.,               2.,                   2,                0 ], # Ampere
        [ 0.,                  0.,                   0.,              0.,                   0.,                 1.,           0.,                0.,                    0.,              0.,        0.,       0.,               0.,                   0,               -1 ], # K
    ],
)

In [14]:
# Experiment Design Data

#units: m s atom kg Amp K

df4_names = ['Volume_rxtor_m3', 'V_all_beads_m2','A_a_m2','A_b_m2', 'h_m', 'Volume_m3', 'A_bead_m2', 'A_tot_m3', 'frequency_Hz', 'Flow_m3_s', 'temp_C_gas_K', 'Set_Voltage_kgm2_s3_A', 'pulse_time_seconds', 'd_bead' ]
D_df_4 = np.array(
    [    #Volume_rxtor_m3  V_all_beads_m2     A_a_m2   A_b_m2      h_m     Volume_m3      A_bead_m2      A_tot_m2    frequency_Hz     Flow_m3_s  temp_C_gas_Kg   Set_Voltage_kgm2_s3_A     pulse_time_seconds      
        [ 3.,                  2.,              2.,     2.,         1.,        3.,           2.,            2.,           0.,            3.,        0.,                  2.,                     0.,              1 ], # m
        [ 0.,                  0.,              0.,     0.,         0.,        0.,           0.,            0.,          -1.,           -1.,        0.,                 -3.,                     1.,              0 ], # s
        [ 0.,                  0.,              0.,     0.,         0.,        0.,           0.,            0.,           0.,            0.,        0.,                  0.,                     0.,              0 ], # atom
        [ 0.,                  0.,              0.,     0.,         0.,        0.,           0.,            0.,           0.,            0.,        0.,                  1.,                     0.,              0 ], # kg
        [ 0.,                  0.,              0.,     0.,         0.,        0.,           0.,            0.,           0.,            0.,        0.,                 -1.,                     0.,              0 ], # Ampere
        [ 0.,                  0.,              0.,     0.,         0.,        0.,           0.,            0.,           0.,            0.,        1.,                  0.,                     0.,              0 ], # K
    ],
)

In [15]:
import pandas as pd
import numpy as np
import os

# File path of the Excel file
file_path = 'data_from_EB_looping_new_excel_new_code_5cm_23mm.xlsx'

# Check if the file exists
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"File '{file_path}' deleted successfully.")
else:
    print(f"File '{file_path}' does not exist.")


#Units on K: 2 reactants are [m3][atoms-1][s-1]; three reactants are [m6][atoms-2][s-1]

df1 = pd.DataFrame(data=array_a[good_index,:],columns=df1_names)
df1_units = pd.DataFrame(data=D_df_1,columns=df1_names)
df1n = pd.DataFrame([df1_names], columns=df1_names)


df2 = pd.DataFrame(data=array_b[good_index,:], columns =df2_names)
df2_units = pd.DataFrame(data=D_df_2,columns=df2_names)
df2n = pd.DataFrame([df2_names], columns=df2_names)



df3 = pd.DataFrame(data=array_all[good_index,:], columns = df3_names)
df3_units = pd.DataFrame(data=D_df_3,columns=df3_names)
df3n = pd.DataFrame([df3_names], columns=df3_names)


#make the minor constant adjustments and rerun in separate sheet
df4 = pd.DataFrame(data=array_geom[good_index,:], columns = df4_names)
df4_units = pd.DataFrame(data=D_df_4,columns=df4_names)
df4n = pd.DataFrame([df4_names], columns=df4_names)


df5 = pd.DataFrame(data=[vec_sign[i] for i in good_index], columns = ['signature'])

df_mb_rates = pd.DataFrame(data=array_mb_rates[good_index,:], columns =mass_balance_rates_names )

df_eb_rates = pd.DataFrame(data=array_eb_rates[good_index,:], columns =energy_balance_rates_names )
    
with pd.ExcelWriter(file_path) as writer:  
    df1.to_excel(writer, sheet_name='time_a_data')
    df1_units.to_excel(writer, sheet_name='time_a_data_units')
    df1n.to_excel(writer, sheet_name='time_a_data_names')
    df2.to_excel(writer, sheet_name='time_b_data')
    df2_units.to_excel(writer, sheet_name='time_b_data_units')
    df2n.to_excel(writer, sheet_name='time_b_data_names')
    df3.to_excel(writer, sheet_name='other_data')
    df3_units.to_excel(writer, sheet_name='other_data_units')
    df3n.to_excel(writer, sheet_name='other_data_names')
    df4.to_excel(writer, sheet_name='Experiment_Design_data')
    df4_units.to_excel(writer, sheet_name='Experiment_Data_units')
    df4n.to_excel(writer, sheet_name='Experiment_Data_names')
    df5.to_excel(writer, sheet_name='signature')
    df_mb_rates.to_excel(writer,sheet_name = 'mb_rates')
    df_eb_rates.to_excel(writer,sheet_name = 'eb_rates')
    
    
    
    
    
file_path = 'mass_balance_params_5cm_23mm.xlsx'

# Check if the file exists
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"File '{file_path}' deleted successfully.")
else:
    print(f"File '{file_path}' does not exist.")


algorithm = array_mb[good_index,:]

outputs = algorithm[:,0]/algorithm[:,1]
inputs = algorithm[:,2:]

df6 = pd.DataFrame(data=outputs, columns = ['n_e_divided_ng'])

df7 = pd.DataFrame(data=inputs, columns = [ 'u_B_m_s', 'A_tot_m2', 't_a_s', 't_b_s', 'Volume_m3', 'K_iz_a_m3_s_atom', 'K_2_iz_a_m3_s_atom', 'K_iz_exc_a_m3_s_atom', 'n_sa_atoms_m3', 'n_sb_atoms_m3', 'A_a_m2', 'A_b_m2', 'n_He_exc_a_atoms_m3'])


with pd.ExcelWriter(file_path) as writer:  
    df6.to_excel(writer, sheet_name='dimensionless_output')
    df7.to_excel(writer, sheet_name='inputs')
    

    

    
    
    
file_path = 'energy_balance_params_5cm_23mm.xlsx'

# Check if the file exists
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"File '{file_path}' deleted successfully.")
else:
    print(f"File '{file_path}' does not exist.")


algorithm = array_eb[good_index,:]

outputs = algorithm[:,0]/algorithm[:,1]
inputs = algorithm[:,2:]

df6 = pd.DataFrame(data=outputs, columns = ['T_e_divided_Tg'])
  

# E [=] J/particle [=] kgm2_s2_particle
# y = K*E [=] m3_s_p * kgm2_s2_p [=] kgm5_s3_p2  this is only true for 2 species reactions (all but K_loss) 
    
df7 = pd.DataFrame(data=inputs, columns = [ 'Volume_m3', 'n_e_atoms_m3', 'n_g_atoms_m3', 'A_a_m2', 'A_b_m2', 'e_c_As', 'A_tot', 'E_kgm2_s2_atom', 'y_exc_a_kgm5_s3_atoms2', 'y_iz_kgm5_s3_atoms2', 'y_iz_exc_kgm5_s3_atoms2', 'y_elastic_kgm5_s3_atoms2', 't_a_s', 'n_sa_atoms_m3', 'n_sb_atoms_m3', 'V_p_ta_volts', 't_b_s', 'u_B_m_s', 'V_p_tb_volts', 'v_e_m_s', 'n_He_exc_a_atoms_m3'  ])


with pd.ExcelWriter(file_path) as writer:  
    df6.to_excel(writer, sheet_name='dimensionless_output')
    df7.to_excel(writer, sheet_name='inputs')



File 'data_from_EB_looping_new_excel_new_code_5cm_23mm_recheck_5_28.xlsx' does not exist.
File 'mass_balance_params_5cm_23mm_recheck_5_28.xlsx' does not exist.
File 'energy_balance_params_5cm_23mm_recheck_5_28.xlsx' does not exist.
